`make download-osm`

In [ ]:
import sys
from pathlib import Path

AVES_ROOT = Path("../..")

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import geopandas as gpd

# esto configura la calidad de la imagen. dependerá de tu resolución. el valor por omisión es 80
mpl.rcParams["figure.dpi"] = 96
# esto depende de las fuentes que tengas instaladas en el sistema.
mpl.rcParams["font.family"] = "Fira Sans Extra Condensed"

In [ ]:
zones = (
    gpd.read_file(AVES_ROOT / "data" / "processed" / "scl_zonas_urbanas.json")
    .set_index("ID")
)
zones.head()

In [ ]:
OSM_PATH = AVES_ROOT / "data" / "external" / "OSM"

osm_clipped_file = OSM_PATH / 'clipped-scl-osm.pbf'

if not osm_clipped_file.exists():
    import os
    bounds = zones.total_bounds
    print(bounds)
    print(f"osmconvert {OSM_PATH / 'chile-latest.osm.pbf'} -b={bounds[0]},{bounds[1]},{bounds[2]},{bounds[3]} -o={osm_clipped_file}")
    os.system(f"osmconvert {OSM_PATH / 'chile-latest.osm.pbf'} -b={bounds[0]},{bounds[1]},{bounds[2]},{bounds[3]} -o={osm_clipped_file}")
else:
    print('data already available :D')

In [ ]:
import pyrosm

osm = pyrosm.OSM(str(osm_clipped_file))

## Puntos de Interés (PoI)

In [ ]:
pois = osm.get_pois()

In [ ]:
pois.sample(5)

In [ ]:
len(pois)

In [ ]:
pois.plot(alpha=0.01)

In [ ]:
from aves.features.osm.pois import categorize_pois

cat_pois = categorize_pois(pois)
cat_pois

In [ ]:
cat_pois.plot(alpha=0.01)

In [ ]:
cat_pois['taxonomy'].value_counts()

In [ ]:
cat_pois[cat_pois['taxonomy'] == 'health'].plot()

In [ ]:
zones[['NOM_COMUNA', 'geometry']]

In [ ]:
cat_pois_area = gpd.sjoin(cat_pois, zones[['NOM_COMUNA', 'geometry']], op='within')
cat_pois_area

In [ ]:
from aves.features.utils import normalize_rows
sns.clustermap(cat_pois_area.groupby(['NOM_COMUNA', 'taxonomy']).size().unstack(fill_value=0).pipe(lambda x: np.log(x + 1)), method='ward')

In [ ]:
from aves.features.utils import standardize_columns
sns.clustermap(cat_pois_area.groupby(['NOM_COMUNA', 'taxonomy']).size().unstack(fill_value=0).pipe(standardize_columns), method='ward', center=0)

In [ ]:
zone_pois = cat_pois_area.groupby(['index_right', 'taxonomy']).size().unstack(fill_value=0).pipe(np.sqrt).pipe(standardize_columns)
zone_pois

In [ ]:
from aves.visualization.figures import figure_from_geodataframe
from aves.visualization.maps import choropleth_map

fig, ax = figure_from_geodataframe(zones, height=9)

choropleth_map(ax, zones.join(zone_pois), 'accommodation', linewidth=0)

In [ ]:
from aves.visualization.figures import small_multiples_from_geodataframe

fig, axes = small_multiples_from_geodataframe(zones, 6, height=8, col_wrap=3)

for ax, col in zip(axes, ['health', 'education', 'convenience', 'professional', 'public_facility', 'entertainment']):
    choropleth_map(ax, zones.join(zone_pois), col, linewidth=0)
    ax.set_title(col)

fig.tight_layout()

In [ ]:
from scipy.stats import entropy

zone_entropy = cat_pois_area.groupby(['index_right', 'taxonomy']).size().unstack(fill_value=0).pipe(np.sqrt).apply(entropy, axis=1).rename('poi_diversity').to_frame().pipe(standardize_columns)

fig, ax = figure_from_geodataframe(zones, height=9)

choropleth_map(ax, zones.join(zone_entropy), 'poi_diversity', linewidth=0, scheme='quantiles')

In [ ]:
network_edges  = osm.get_network(network_type='driving')

In [ ]:
network_edges.columns

In [ ]:
network_edges['access'].value_counts()

In [ ]:
network_edges[network_edges['highway'] == 'motorway'].plot()

In [ ]:
fig, ax = figure_from_geodataframe(zones, height=12)

choropleth_map(ax, zones.join(zone_entropy), 'poi_diversity', linewidth=0, scheme='quantiles')

network_edges[network_edges['highway'] == 'motorway'].plot(color='black', ax=ax, linewidth=0.5)
network_edges[network_edges['highway'] == 'primary'].plot(color='grey', ax=ax, linewidth=0.5)

In [ ]:
rail_network = osm.get_data_by_custom_criteria(
            custom_filter=dict(railway=["subway", "usage", "rail"]), osm_keys_to_keep=["railway"], filter_type="keep"
        )

rail_network

In [ ]:
import json

rail_network = rail_network.join(pd.DataFrame.from_records(rail_network["tags"].map(json.loads)))
rail_network.sample(5)

In [ ]:
rail_network.plot()

In [ ]:
rail_network['name'].value_counts()

In [ ]:
rail_network[rail_network['name'].fillna('').str.contains('Línea|Metro')].plot()

In [ ]:
fig, ax = figure_from_geodataframe(zones, height=12)

choropleth_map(ax, zones.join(zone_entropy), 'poi_diversity', linewidth=0, scheme='quantiles')

network_edges[network_edges['highway'] == 'motorway'].plot(color='black', ax=ax, linewidth=0.5)
network_edges[network_edges['highway'] == 'primary'].plot(color='grey', ax=ax, linewidth=0.5)

rail_network[rail_network['name'].fillna('').str.contains('Línea|Metro')].plot(color='purple', ax=ax, linewidth=2)